# Stats in Paper

In [1]:
# !pip install ipywidgets

In [9]:
# imports
import ipdb, os, re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from tqdm import tqdm

### Selected Dataset

In [2]:
# New_train_csv = "/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/pwc_ibm_150_5_10_10000/10Neg10000unk/twofoldwithunk/fold1/train.tsv"
# New_test_csv = "/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/pwc_ibm_150_5_10_10000/10Neg10000unk/twofoldwithunk/fold1/dev.tsv"

# New_trainOutput_csv = "/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/pwc_ibm_150_5_10_10000/10Neg10000unk/trainOutput.tsv"

# IBM_train_csv = "~/Research/task-dataset-metric-nli-extraction/data/ibm/exp/few-shot-setup/NLP-TDMS/paperVersion/train.tsv"
# IBM_test_csv = "~/Research/task-dataset-metric-nli-extraction/data/ibm/exp/few-shot-setup/NLP-TDMS/paperVersion/test.tsv"

# Old_train_csv = "/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/other/pwc_ibm_150_5_10_800/twofoldwithunk/fold1/train.tsv"
# Old_test_csv = "/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/other/pwc_ibm_150_5_10_800/twofoldwithunk/fold1/dev.tsv"

# IBM_train_csv = "/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/ibm/exp/zero-shot-setup/NLP-TDMS/train.tsv"
# IBM_test_csv = "/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/ibm/exp/zero-shot-setup/NLP-TDMS/test.tsv"

# # New_train_csv = IBM_train_csv
# # New_test_csv = IBM_test_csv

# # New_train_csv =Old_train_csv
# # New_test_csv = Old_test_csv

In [3]:
zero_shot = "/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/zero-shot/pwc_pdf_5_10_800/twofoldwithunk/fold1"
pwc_ibm_150_5_10_10000 = "/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/pwc_ibm_150_5_10_10000/10Neg10000unk/twofoldwithunk/fold1"

In [4]:
train_zero_shot = pd.read_csv(f"{zero_shot}/train.tsv", 
                    sep="\t", names=["label", "title", "TDM", "Context"])

test_zero_shot = pd.read_csv(f"{zero_shot}/dev.tsv", 
                    sep="\t", names=["label", "title", "TDM", "Context"])

train_pwc_ibm_150_5_10_10000 = pd.read_csv(f"{pwc_ibm_150_5_10_10000}/train.tsv", 
                    sep="\t", names=["label", "title", "TDM", "Context"])

test_pwc_ibm_150_5_10_10000 = pd.read_csv(f"{pwc_ibm_150_5_10_10000}/dev.tsv", 
                    sep="\t", names=["label", "title", "TDM", "Context"])

In [5]:
train_pwc_ibm_150_5_10_10000.head()

,label,title,TDM,Context
0,True,1707.03497v2.pdf,Atari Games; Atari 2600 Seaquest; Score,Value Prediction Network This paper proposes a...
1,True,1707.03497v2.pdf,Atari Games; Atari 2600 Amidar; Score,Value Prediction Network This paper proposes a...
2,True,1707.03497v2.pdf,Atari Games; Atari 2600 Krull; Score,Value Prediction Network This paper proposes a...
3,True,1707.03497v2.pdf,Atari Games; Atari 2600 Alien; Score,Value Prediction Network This paper proposes a...
4,True,1707.03497v2.pdf,Atari Games; Atari 2600 Enduro; Score,Value Prediction Network This paper proposes a...


In [6]:
tdm_count = defaultdict(lambda: 0)
for tdm in train_zero_shot[train_zero_shot.label==True].TDM.to_list():
    tdm_count[tdm] +=1

In [7]:
# tdm_count

In [8]:
train_tdm_taxonomy = set(train_pwc_ibm_150_5_10_10000[train_pwc_ibm_150_5_10_10000.label==True].TDM.to_list())

In [9]:
train_zero_shot = train_zero_shot[train_zero_shot.label==True]
train_zero_shot.head()

,label,title,TDM,Context
10,True,2006.10721v2.pdf,Visual Object Tracking; GOT-10k; Average Overlap,Ocean: Object-aware Anchor-free Tracking Ancho...
11,True,2006.10721v2.pdf,Visual Object Tracking; GOT-10k; Success Rate 0.5,Ocean: Object-aware Anchor-free Tracking Ancho...
22,True,2210.11416v2.pdf,Multi-task Language Understanding; MMLU; Avera...,Scaling Instruction-Finetuned Language Models ...
23,True,2210.11416v2.pdf,Multi-task Language Understanding; MMLU; Param...,Scaling Instruction-Finetuned Language Models ...
34,True,2005.02190v2.pdf,Action Anticipation; EPIC-KITCHENS-100 (test);...,Rolling-Unrolling LSTMs for Action Anticipatio...


In [10]:
# train_zero_shot_final = train_zero_shot[train_zero_shot.TDM in train_tdm_taxonomy]

In [14]:
df = pd.DataFrame(columns = ["label", "title", "TDM", "Context"])
for index, row in train_zero_shot.iterrows():
    # ipdb.set_trace()
    if row["TDM"] not in train_tdm_taxonomy:
        df = pd.concat([df, pd.DataFrame.from_records([{'label' : row.label, 'title' : row.title, 'TDM' : row.TDM, 'Context' : row.Context}])], ignore_index = True)
        # df = df.append({'label' : row.label, 'title' : row.title, 'TDM' : row.TDM, 'Context' : row.Context},
        #                ignore_index = True)

In [16]:
df.tail()

,label,title,TDM,Context
8892,True,2007.08176v2.pdf,Anomaly Detection; Anomaly Detection on Anomal...,CSI: Novelty Detection via Contrastive Learnin...
8893,True,2007.08176v2.pdf,Anomaly Detection; Anomaly Detection on Unlabe...,CSI: Novelty Detection via Contrastive Learnin...
8894,True,2007.08176v2.pdf,Anomaly Detection; Anomaly Detection on Unlabe...,CSI: Novelty Detection via Contrastive Learnin...
8895,True,1505.02419v3.pdf,Relation Extraction; ACE 2005; Cross Sentence,Improved Relation Extraction with Feature-Rich...
8896,True,1912.04799v2.pdf,Monocular 3D Object Detection; KITTI Cars Mode...,Learning Depth-Guided Convolutions for Monocul...


In [17]:
df.head()

,label,title,TDM,Context
0,True,2210.11416v2.pdf,Multi-task Language Understanding; MMLU; Avera...,Scaling Instruction-Finetuned Language Models ...
1,True,2210.11416v2.pdf,Multi-task Language Understanding; MMLU; Param...,Scaling Instruction-Finetuned Language Models ...
2,True,2005.02190v2.pdf,Action Anticipation; EPIC-KITCHENS-100 (test);...,Rolling-Unrolling LSTMs for Action Anticipatio...
3,True,2202.11539v2.pdf,Weakly-Supervised Object Localization; CUB-200...,SELF-SUPERVISED TRANSFORMERS FOR UNSUPERVISED ...
4,True,2202.11539v2.pdf,Single-object discovery; COCO_20k; CorLoc,SELF-SUPERVISED TRANSFORMERS FOR UNSUPERVISED ...


In [18]:
df.describe()

,label,title,TDM,Context
count,8897,8897,8897,8897
unique,1,2688,1872,2687
top,True,2108.10904v3.pdf,Image Classification; ImageNet; GFLOPs,SIMVLM: SIMPLE VISUAL LANGUAGE MODEL PRE- TRAI...
freq,8897,52,98,52


In [19]:
output_path = "/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/zero-shot/clearned"
df.to_csv(f"{output_path}/train.tsv", 
                       header=False, index=False, sep="\t")

In [20]:
df['Count'] = df.TDM.apply(lambda x: tdm_count[x])

In [21]:
# df[df['Count'] >=5]
df[df['Count'] ==0]

,label,title,TDM,Context,Count


In [22]:
df.head()

,label,title,TDM,Context,Count
0,True,2210.11416v2.pdf,Multi-task Language Understanding; MMLU; Avera...,Scaling Instruction-Finetuned Language Models ...,8
1,True,2210.11416v2.pdf,Multi-task Language Understanding; MMLU; Param...,Scaling Instruction-Finetuned Language Models ...,7
2,True,2005.02190v2.pdf,Action Anticipation; EPIC-KITCHENS-100 (test);...,Rolling-Unrolling LSTMs for Action Anticipatio...,3
3,True,2202.11539v2.pdf,Weakly-Supervised Object Localization; CUB-200...,SELF-SUPERVISED TRANSFORMERS FOR UNSUPERVISED ...,4
4,True,2202.11539v2.pdf,Single-object discovery; COCO_20k; CorLoc,SELF-SUPERVISED TRANSFORMERS FOR UNSUPERVISED ...,5


In [23]:
df.loc[0, "TDM"]

'Multi-task Language Understanding; MMLU; Average (%)'

## Merge TdmGoldLabels

In [24]:
zero_shot_gold_tdm = "/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/zero-shot/pwc_pdf_5_10_800"
pwc_ibm_150_5_10_10000_gold_tdm = "/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/pwc_ibm_150_5_10_10000/10Neg10000unk"

In [25]:
pd_zero_shot_gold_tdm = pd.read_csv(f"{zero_shot_gold_tdm}/tdmGoldLabels.tsv", 
                    sep="\t", names=["tdm"])

pd_pwc_ibm_150_5_10_10000_gold_tdm = pd.read_csv(f"{pwc_ibm_150_5_10_10000_gold_tdm}/tdmGoldLabels.tsv", 
                    sep="\t", names=["tdm"])

In [26]:
pd_zero_shot_gold_tdm.head()

,tdm
0,Single Image Deraining#Rain100H#PSNR
1,Question Answering#YahooCQA#P@1
2,Atari Games#Atari 2600 Private Eye#Score
3,Crowd Counting#WorldExpo???10#Average MAE
4,Speech Recognition#MediaSpeech#WER for Turkish


In [27]:
pd_zero_shot_gold_tdm.describe()

,tdm
count,3551
unique,3551
top,Single Image Deraining#Rain100H#PSNR
freq,1


In [28]:
pd_pwc_ibm_150_5_10_10000_gold_tdm.describe()

,tdm
count,1851
unique,1851
top,Single Image Deraining#Rain100H#PSNR
freq,1


In [30]:
df_merge_gold = pd.DataFrame(columns = ["tdm"])
set_merge_gold = set()
for index, row in pd_zero_shot_gold_tdm.iterrows():
    # ipdb.set_trace()
    if row["tdm"] not in set_merge_gold:
        # df_merge_gold = df_merge_gold.append({'tdm' : row.tdm},
        #                ignore_index = True)
        df_merge_gold = pd.concat(
            [df_merge_gold, pd.DataFrame.from_records([{'tdm' : row.tdm}])], ignore_index = True)
        
        set_merge_gold.add(row["tdm"])

for index, row in pd_pwc_ibm_150_5_10_10000_gold_tdm.iterrows():
    # ipdb.set_trace()
    if row["tdm"] not in set_merge_gold:
        # df_merge_gold = df_merge_gold.append({'tdm' : row.tdm},
        #                ignore_index = True)
        df_merge_gold = pd.concat(
            [df_merge_gold, pd.DataFrame.from_records([{'tdm' : row.tdm}])], ignore_index = True)
        
        set_merge_gold.add(row["tdm"])

In [31]:
df_merge_gold.describe()

,tdm
count,3741
unique,3741
top,Single Image Deraining#Rain100H#PSNR
freq,1


In [32]:
output_path = "/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/zero-shot/clearned"
df_merge_gold.to_csv(f"{output_path}/tdmGoldLabels.tsv", 
                       header=False, index=False, sep="\t")

## Create Zero-shot test

In [3]:
output_path = "/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/zero-shot/clearned"

df = pd.read_csv(f"{output_path}/train.tsv", 
                    sep="\t", names=["label", "title", "TDM", "Context"])

df_merge_gold= pd.read_csv(f"{output_path}/tdmGoldLabels.tsv", 
                       sep="\t", names=["tdm"])

In [4]:
df.tail()

,label,title,TDM,Context
8892,True,2007.08176v2.pdf,Anomaly Detection; Anomaly Detection on Anomal...,CSI: Novelty Detection via Contrastive Learnin...
8893,True,2007.08176v2.pdf,Anomaly Detection; Anomaly Detection on Unlabe...,CSI: Novelty Detection via Contrastive Learnin...
8894,True,2007.08176v2.pdf,Anomaly Detection; Anomaly Detection on Unlabe...,CSI: Novelty Detection via Contrastive Learnin...
8895,True,1505.02419v3.pdf,Relation Extraction; ACE 2005; Cross Sentence,Improved Relation Extraction with Feature-Rich...
8896,True,1912.04799v2.pdf,Monocular 3D Object Detection; KITTI Cars Mode...,Learning Depth-Guided Convolutions for Monocul...


In [5]:
df_merge_gold["tdm"] = df_merge_gold.tdm.apply(lambda x: x.replace("#", "; "))

In [6]:
df_merge_gold.head()

,tdm
0,Single Image Deraining; Rain100H; PSNR
1,Question Answering; YahooCQA; P@1
2,Atari Games; Atari 2600 Private Eye; Score
3,Crowd Counting; WorldExpo???10; Average MAE
4,Speech Recognition; MediaSpeech; WER for Turkish


In [7]:
# gold_tdm_list
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [ ]:
zero_shot_title = set(df.title.to_list())

df_test = pd.DataFrame(columns = ["label", "title", "TDM", "Context"])
set_test = set()

for title in tqdm(zero_shot_title):
    gold_tdm_list = df_merge_gold.tdm.to_list()
    row =""
    for index, row in df[df.title == title].iterrows():
        # df_test = df_test.append({'label' : row.label, 'title' : row.title, 'TDM' : row.TDM, 'Context' : row.Context},
        #                          ignore_index = True)
        df_test = pd.concat(
            [df_test, pd.DataFrame.from_records(
                [{'label' : row.label, 'title' : row.title, 'TDM' : row.TDM, 'Context' : row.Context}])], ignore_index = True)
        
        gold_tdm_list.remove(row.TDM)
    
    for tdm in gold_tdm_list:
        # df_test = df_test.append({'label' : False, 'title' : row.title, 'TDM' : tdm, 'Context' : row.Context},
        #                          ignore_index = True)
        df_test = pd.concat(
            [df_test, pd.DataFrame.from_records(
                [{'label' : False, 'title' : row.title, 'TDM' : tdm, 'Context' : row.Context}])], ignore_index = True)

 33%|██████████████████████████████████████████████▊                                                                                                | 879/2688 [41:28:34<176:15:33, 350.76s/it]

In [ ]:
df_test.head()

In [5]:
train_unique_tdm = set(train_New[train_New.label==True].TDM.to_list())

In [6]:
test_unique_tdm = set(test_New[test_New.label==True].TDM.to_list())

In [7]:
print(f"unique TDM train: {len(train_unique_tdm)}")
print(f"unique TDM test: {len(test_unique_tdm)}")

unique TDM train: 1792
unique TDM test: 1557


In [8]:
def unique_tdm_check(left, right, mode="Train"):
    count = 0
    count_set = set()
    for tdm in left:
        if tdm not in right:
            count_set.add(tdm)
            count  += 1
    print(f"Missing TDM {mode}: {count}")
    return count_set

## Create Zero-shoot

In [9]:
def create_zero_shoot(train, test, output_path="/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/pwc_ibm_150_5_10_500/10Neg500unk_shot/zero_shot_twofoldwithunk/fold1/"):
    
    if not os.path.exists(output_path):
        os.makedirs(output_path)
        
    unique_sets_train = set(train[train.label==True].TDM.to_list())
    unique_sets_test = set(test[test.label==True].TDM.to_list())
        
    print(f"Len unique TDM Train: {len(unique_sets_train)}.")
    print(f"Len unique TDM Test: {len(unique_sets_test)}.")
    
    print(f"Size Train df before: {len(train)}.")
    print(f"Size Train df before: {len(test)}.")
    
    for label in unique_sets_train:
        index_to_drop = test[(test.TDM ==label) & (test.label==True)].index
        test.drop(index_to_drop, axis=0, inplace=True)
        
    for label in unique_sets_test:
        index_to_drop = train[(train.TDM ==label) & (train.label==True)].index
        train.drop(index_to_drop, axis=0, inplace=True)
        
    train.to_csv(f"{output_path}train.tsv", 
                              header=False, index=False, sep="\t")    
    test.to_csv(f"{output_path}dev.tsv", 
                              header=False, index=False, sep="\t")
    
    unique_sets_train = set(train[train.label==True].TDM.to_list())
    unique_sets_test = set(test[test.label==True].TDM.to_list())
    
    print(f"Len unique TDM Train: {len(unique_sets_train)}.")
    print(f"Len unique TDM Test: {len(unique_sets_test)}.")
    
    print(f"Size Train df After: {len(train)}.")
    print(f"Size Test df After: {len(test)}.")
    
    print("Done.")

In [10]:
# def create_zero_shoot(trainOutput_New, split=0.8, output_path="/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/pwc_ibm_150_5_10_500/10Neg500unk_shot/zero_shot_twofoldwithunk/fold1/"):
    
#     if not os.path.exists(output_path):
#         os.makedirs(output_path)
        
#     unique_sets = set(trainOutput_New[trainOutput_New.label==True].TDM.to_list())
#     train_split_tdm, test_split_tdm = train_test_split(list(unique_sets), train_size=split)
    
#     print(f"Len unique TDM {len(unique_sets)}.")
    
#     Zero_shoot_train_New = pd.DataFrame().reindex_like(trainOutput_New).dropna()
#     Zero_shoot_test_New = pd.DataFrame().reindex_like(trainOutput_New).dropna()
    
#     for label in train_split_tdm:
#         Zero_shoot_train_New = Zero_shoot_train_New.merge(trainOutput_New[(trainOutput_New.TDM==label) & (trainOutput_New.TDM==label)], how='outer')
        
#     for label in test_split_tdm:
#         Zero_shoot_test_New = Zero_shoot_test_New.merge(trainOutput_New[trainOutput_New.TDM==label], how='outer')
        
#     Zero_shoot_train_New.to_csv(f"{output_path}train.tsv", 
#                               header=False, index=False, sep="\t")
    
#     print(f"Len train dataset {len(Zero_shoot_train_New)}.")
    
#     Zero_shoot_test_New.to_csv(f"{output_path}dev.tsv", 
#                               header=False, index=False, sep="\t")
    
#     print(f"Len test dataset {len(Zero_shoot_test_New)}.")
    
#     print("Done.")

In [11]:
output = "/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/pwc_ibm_150_5_10_10000/10Neg10000unk/zero_shot_2_twofoldwithunk/fold1/"

In [12]:
# create_zero_shoot(trainOutput_New=trainOutput_New, split=0.8, 
#                   output_path=output)

In [13]:
create_zero_shoot(train=train_New, test=test_New, 
                  output_path=output)

Len unique TDM Train: 1792.
Len unique TDM Test: 1557.
Size Train df before: 52154.
Size Train df before: 22426.
Len unique TDM Train: 294.
Len unique TDM Test: 59.
Size Train df After: 38608.
Size Test df After: 16165.
Done.


In [14]:
tdm_in_test_not_train = unique_tdm_check(left=New_test_csv, right=New_train_csv, mode="Test")

Missing TDM Test: 0


In [15]:
tdm_in_train_not_test = unique_tdm_check(left=New_train_csv, right=New_test_csv, mode="Train")

Missing TDM Train: 0


In [16]:
New_train_csv = "/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/pwc_ibm_150_5_10_10000/10Neg10000unk/twofoldwithunk/fold2/train.tsv"
New_test_csv = "/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/pwc_ibm_150_5_10_10000/10Neg10000unk/twofoldwithunk/fold2/dev.tsv"

train_New = pd.read_csv(New_train_csv, 
                    sep="\t", names=["label", "title", "TDM", "Context"])

test_New = pd.read_csv(New_test_csv, 
                    sep="\t", names=["label", "title", "TDM", "Context"])

In [17]:
output = "/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/pwc_ibm_150_5_10_10000/10Neg10000unk/zero_shot_2_twofoldwithunk/fold2/"

In [18]:
# create_zero_shoot(trainOutput_New=trainOutput_New, split=0.8, 
#                   output_path=output)
create_zero_shoot(train=train_New, test=test_New, 
                  output_path=output)

Len unique TDM Train: 1821.
Len unique TDM Test: 1542.
Size Train df before: 52315.
Size Train df before: 22265.
Len unique TDM Train: 309.
Len unique TDM Test: 30.
Size Train df After: 38649.
Size Test df After: 16131.
Done.


In [19]:
New_train_csv = f"{output}train.tsv"
New_test_csv = f"{output}dev.tsv"

In [20]:
tdm_in_test_not_train = unique_tdm_check(left=New_test_csv, right=New_train_csv, mode="Test")

Missing TDM Test: 0


In [21]:
tdm_in_train_not_test = unique_tdm_check(left=New_train_csv, right=New_test_csv, mode="Train")

Missing TDM Train: 0


## Enforce seen TDM in both train and test

In [101]:
print(f"unique TDM train: {len(train_unique_tdm)}")
print(f"unique TDM test: {len(test_unique_tdm)}")

unique TDM train: 96
unique TDM test: 63


In [97]:
tdm_in_test_not_train = unique_tdm_check(left=test_unique_tdm, right=train_unique_tdm, mode="Test")

Missing TDM Test: 63


In [98]:
tdm_in_train_not_test = unique_tdm_check(left=train_unique_tdm, right=test_unique_tdm, mode="Train")

Missing TDM Train: 96


In [99]:
len(train_New)

20160

In [100]:
len(test_New)

17388

In [111]:
test_New[(test_New.TDM=="Fact-based Text Editing; WebEdit; ADD") & (test_New.label==True)]

,label,title,TDM,Context
14199,True,2007.00916v1.pdf,Fact-based Text Editing; WebEdit; ADD,Fact-based Text Editing We propose a novel tex...


In [112]:
train_New[(train_New.TDM=="Fact-based Text Editing; WebEdit; ADD") & (train_New.label==True)]

,label,title,TDM,Context
